In [17]:
#아이템 데이터
items = [['A', 'B', 'C'], ['A', 'B'], ['A', 'C'], ['B', 'C'],
        ['A', 'B', 'C'], ['B', 'C'], ['A', 'C'], ['A', 'B'], 
         ['A'], ['A', 'C']]
print(items)

#최소 지지도 임계치 설정
min_support = 0.3 #이 임계치 이상의 지지도를 가지는 아이템을 찾을 것

#아이템 지지도 계산
item_counts = {}
transaction = len(items) #전체 트랜잭션의 수
print(transaction)

#각 트랜잭션에서 아이템의 지지도를 계산
for i in items:
#     print(i)
    for j in i:
#         print(j)
        if j in item_counts:
            item_counts[j] += 1
        else:
            item_counts[j] = 1
            
print(item_counts)

#아이템 지지도가 최소 지지도 임계치 이상인 아이템들을 찾기
frequent = {}

for item, support in item_counts.items():
#     print(item, support)
    if support / transaction >= min_support:
        frequent[item] = support

print("아이템 지지도")
print(frequent)


[['A', 'B', 'C'], ['A', 'B'], ['A', 'C'], ['B', 'C'], ['A', 'B', 'C'], ['B', 'C'], ['A', 'C'], ['A', 'B'], ['A'], ['A', 'C']]
10
{'A': 8, 'B': 6, 'C': 7}
아이템 지지도
{'A': 8, 'B': 6, 'C': 7}


# apriori 알고리즘

In [19]:
!pip install mlxtend

  Obtaining dependency information for mlxtend from https://files.pythonhosted.org/packages/1c/07/512f6a780239ad6ce06ce2aa7b4067583f5ddcfc7703a964a082c706a070/mlxtend-0.23.1-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
    --------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---- ----------------------------------- 0.2/1.4 MB 3.5 MB/s eta 0:00:01
   ------------- -------------------------- 0.5/1.4 MB 6.2 MB/s eta 0:00:01
   ------------------------ --------------- 0.9/1.4 MB 7.0 MB/s eta 0:00:01
   ----------------------------------- ---- 1.3/1.4 MB 8.2 MB/s eta 0:00:01
   ---------------------------------------- 1.4/1.4 MB 8.4 MB/s eta 0:00:00


In [20]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import pandas as pd

In [21]:
items = [['A', 'B', 'C'], ['A', 'B'], ['A', 'C'], ['B', 'C'],
        ['A', 'B', 'C'], ['B', 'C'], ['A', 'C'], ['A', 'B'], 
         ['A'], ['A', 'C']]

df = pd.DataFrame(items, columns = ['Item1', 'Item2', 'Item3'])
df

,Item1,Item2,Item3
0,A,B,C
1,A,B,None
2,A,C,None
3,B,C,None
4,A,B,C
5,B,C,None
6,A,C,None
7,A,B,None
8,A,None,None
9,A,C,None


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Item1   10 non-null     object
 1   Item2   9 non-null      object
 2   Item3   2 non-null      object
dtypes: object(3)
memory usage: 372.0+ bytes


In [25]:
#원핫인코딩
one_hot = pd.get_dummies(df)
one_hot

,Item1_A,Item1_B,Item2_B,Item2_C,Item3_C
0,True,False,True,False,True
1,True,False,True,False,False
2,True,False,False,True,False
3,False,True,False,True,False
4,True,False,True,False,True
5,False,True,False,True,False
6,True,False,False,True,False
7,True,False,True,False,False
8,True,False,False,False,False
9,True,False,False,True,False


In [28]:
#빈발항목 집합 찾기
frequent = apriori(one_hot, min_support = 0.3, use_colnames = True)
frequent
#one_hot : 분석할 데이터셋
#min_suppport : 최소 지지도(이 값보다 높은 지지도를 가진 항목 집합만 반환)
#use_colnames : 반환된 데이터프레임에서 항목 집합을 열이름으로 사용할지 결정


,support,itemsets
0,0.8,(Item1_A)
1,0.4,(Item2_B)
2,0.5,(Item2_C)
3,0.4,"(Item2_B, Item1_A)"
4,0.3,"(Item2_C, Item1_A)"


In [35]:
#연관규칙 추출
#모든 연관규칙을 추출하려면 최소 임계치를 0으로 설정한다
rules = association_rules(frequent, metric = 'lift', min_threshold= 0.0)
rules

#frequent : 빈발항목집합 데이터프레임(apriori 생성)
#metric : 규칙의 품질을 측정하는 평가지표
#lift, upport, confidence, leverage, confiction
#min_threshold : 최소 임계치(기본값 1.0)

#데이터프레임 반환
#antecedents : 규칙의 전제항목
#consequents : 규칙의 결과항목
#support : 규칙의 지지도를 나타냄
#confidence : 규칙의 신뢰도
#lift : 규칙의 리프트(독립적인 확률에 비해 규칙을 따를 확률의 상대적 증가)
#antecedent support : 규칙의 전제항목의 지지도, 전체데이터셋에서 전제항목이 발생할 비율
#consequent support : 규칙의 결과항목의 지지도, 전체데이터셋에서 결과항목이 발생할 비율
#support :  전체데이터셋에서 전제항목과 결과항목이 동시에 발생할 비율

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(Item2_B),(Item1_A),0.4,0.8,0.4,1.000,1.25,0.08,inf,0.333333
1,(Item1_A),(Item2_B),0.8,0.4,0.4,0.500,1.25,0.08,1.2,1.000000
2,(Item2_C),(Item1_A),0.5,0.8,0.3,0.600,0.75,-0.10,0.5,-0.400000
3,(Item1_A),(Item2_C),0.8,0.5,0.3,0.375,0.75,-0.10,0.8,-0.625000


In [33]:
rules['antecedents'] #조건절

0    (Item2_B)
1    (Item1_A)
Name: antecedents, dtype: object